<a href="https://colab.research.google.com/github/gordeli/NLP_EDHEC/blob/main/02_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initial Setup

- **Run "Setup" below first.**

    - This will load libraries and download some resources that we'll use throughout the tutorial.

    - You will see a message reading "Done with setup!" when this process completes.



In [ ]:
#@title Setup (click the "run" button to the left) {display-mode: "form"}

## Setup ##

# imports

# built-in Python libraries
# -------------------------

# For processing the incoming Twitter data
import json
# os-level utils
import os
# For downloading web data
import requests
# For compressing files
import zipfile

# 3rd party libraries
# -------------------

# beautiful soup for html parsing
!pip install beautifulsoup4
import bs4

# tweepy for using the Twitter API
!pip install tweepy
import tweepy


# allows downloading of files from colab to your computer
from google.colab import files

# get sample reddit data
if not os.path.exists("reddit_2019_05_5K.json"):
    !wget https://raw.githubusercontent.com/gordeli/textanalysis/master/data/reddit_2019_05_5K.json


print()
print("Done with setup!")
print("If you'd like, you can click the (X) button to the left to clear this output.")

---

## Data Collection

- Here we'll cover a few different sources of user-generated content and provide some examples of how to gather data.

### Web Scraping and HTML parsing

- Lots of text data is available directly from web pages.
- Have a look at the following website: [Quotes to Scrape](http://quotes.toscrape.com/page/1/)
- With the Beautiful Soup library, it's very easy to take some html and extract only the text:

In [ ]:
html_content = requests.get("http://quotes.toscrape.com/page/1/").content
soup = bs4.BeautifulSoup(html_content,"html.parser")
print(soup.text)

- If you want to extract data in a more targeted way, you can navitage the [html document object model](https://www.w3schools.com/whatis/whatis_htmldom.asp) using [Beautiful Soup functions](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), but we won't dive deeply into this for now,
- **Important: You should not use this kind of code to just go collect data from any website!**
    - Web scaping tools should always check a site's [`robots.txt` file](https://www.robotstxt.org/robotstxt.html), which describes how crawlers, scrapers, indexers, etc., should use the site.
        - For example, see [github's robots.txt](https://github.com/robots.txt)
    - You should be able to find any site's robots.txt (if there is one) at http://\<domain\>/robots.txt for any web \<domain\>.

### Reddit Corpus

- Reddit is a great source of publicly available user-generated content.
- We could scrape Reddit ourselves, but why do that if someone has already (generously) done the heavy lifting?
- Reddit user Stuck_in_the_Matrix has compiled and compressed essentially all of Reddit for researchers to download.
- [Original submissions corpus](https://www.reddit.com/r/datasets/comments/3mg812/full_reddit_submission_corpus_now_available_2006/) (up to 2015) and [updates](https://files.pushshift.io/reddit/submissions/) (up to April 2020 at the time of latest update of this notebook).
    - For a smaller file to get started with, take a look at the [daily comments files](https://files.pushshift.io/reddit/comments/daily/).
    - To explore more files available, see [this top-level directory](https://files.pushshift.io/reddit/).
- Let's explore a small subset of the data from May 2019:

In [ ]:
# read the data that was downloaded during setup
# this is the exact format as the full corpus, just truncated to the first 5000 lines
sample_reddit_posts_raw = open("reddit_2019_05_5K.json",'r').readlines()
print("Loaded",len(sample_reddit_posts_raw),"reddit posts.")
reddit_json = [json.loads(post) for post in sample_reddit_posts_raw]
print(json.dumps(reddit_json[50], sort_keys=True, indent=4))

- Since the posts are in json format, we used the Python json library to process them.
    - This library returns Python dict objects, so we can access them just like we would any other dictionary.
- Let's view some of the text content from these posts:

In [ ]:
for post in reddit_json[:100]:
    if post['selftext'].strip() and post['selftext'] not in ["[removed]","[deleted]"]:
        print("Subreddit:",post['subreddit'],"\nTitle:",post['title'],"\nContent:", \
              post['selftext'],"\n")

- Note that we filtered out posts with no text content.
    - Many posts have a non-null "media" field, which could contain images, links to youtube, videos, etc.
        - These could be worth exploring more, using computer vision to process images/videos and NLP to process linked websites.
- That covers the basics of getting Reddit data.

### The Twitter API 

- Twitter is also known for being an abundant source of publc text data (perhaps even more so than Reddit).
- Twitter provides several types of API that can be used to collect anything from tweets to user descriptions to follower networks.
    - You can [read all about it here](https://developer.twitter.com/).
- For this tutorial, we'll look at using the [standard search API](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets.html), which allows us to retreive tweets that contain specific words, phrases, and hashtags.
- In the slides, we talked about how to setup a Twitter App and get a API keys.
    - You should add your own keys below and then run the code block to set your keys:

In [ ]:
twitter_API_key = ""
twitter_API_secret_key = ""

- Do not share your credentials with anyone!
    - You shouldn't hardcode your API keys in code (like above) if you are going to save the file anywhere that is visible to others (like commiting the file to github).
        - You can read more about securing your API keys [here](https://developer.twitter.com/en/docs/basics/authentication/guides/securing-keys-and-tokens).
     - So, if you plan to save this file in any way, make sure to remove your API keys first.
     - If you think your keys have been compromized, you can regenerate them.
        - [Apps](https://developer.twitter.com/en/apps) -> Keys and Tokens -> Regenerate
- Now, let's see how we can use the [tweepy](https://github.com/tweepy/tweepy) library to collect some tweets:

In [ ]:
# create an auth handler object using the api tokens
auth = tweepy.AppAuthHandler(twitter_API_key, twitter_API_secret_key)

# tweepy automatically takes care of potential rate limiting issues
API = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# let's look for some tweets
query = "#Apple"

# count: 100 is the max allowed value for this parameter
#     though we might get fewer than that
# tweet_mode: Twitter changed the char limit from 140->280, but didn't want
#     to break applications expecting 140, so we have to make sure to ask for this.
tweets = API.search(q=query, count=100, tweet_mode="extended")

print("Collected",len(tweets),"tweets.")

- Great, hopefully you got some tweets! Let's take a look:

In [ ]:
print(json.dumps(tweets[0]._json, sort_keys=True, indent=4))

- Here is the text portion of the tweets:

In [ ]:
print("\n\n\n".join([tweet.full_text for tweet in tweets]))

- Things are starting to look a bit more like our examples from the noisy text section.
- Note: retweets are cut off with ... Retweets have 2 full_text fields, one may get the other one properly addressing it based on Json if needed
- To make it even easier to collect tweets from page to page, we can use the tweepy Cursor object:

In [ ]:
cursor = tweepy.Cursor(API.search, q="#EDHEC", tweet_mode="extended")

# just get 5 tweets
# if not given, will (in theory) retrieve as many matching tweets as possible
# (standard search only allows search within previous ~1 week)
for tweet in cursor.items(5):
    print(tweet.full_text)
    print("--------")

### Putting it together: building your own corpus

**Exercise 4:** Tweet collection

- Let's write a function to collect a larger set of tweets related to a query
    - If you want to collect data using multiple queries, you can just call this function multiple times, changing the query each time.
    - Store the tweets in the file howerever you like
        - You will need to write your own parser for this file later on in the tutorial.
    - Store whatever information you like about each tweet, but collect the `full_text` at the very least.
    - Make sure to check if `limit` is set, and if it is, only collect `limit` tweets.

In [ ]:
def write_tweets_to_file(API, query, output_filename, limit=5):
# ------------- Exercise 3 -------------- #
    # gather tweets here, then write to output_filename
# ---------------- End ------------------ #

# quick test
query = "#twitter"
auth = tweepy.AppAuthHandler(twitter_API_key, twitter_API_secret_key)
API = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
output_filename = "test.txt"
write_tweets_to_file(auth, query, output_filename, limit=3)
print("Wrote this to the file:",'\n'+open(output_filename).read())

- Now, change the `query` string below to whatever you like, and run the code.
    - *Make sure your code above is working before you run this! Otherwise, you may run quite a few queries and hit your rate limit, preventing you from testing your code again for ~15 minutes*
    - See [this page](https://developer.twitter.com/en/docs/tweets/search/guides/standard-operators.html) under "standard search operators" for details on what kinds of things you can place here.

In [ ]:
query = "#Apple"

# call the tweet collection function
auth = tweepy.AppAuthHandler(twitter_API_key, twitter_API_secret_key)
API = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
output_filename = "mytweets.txt"
write_tweets_to_file(API, query, output_filename, 1000)

# zip and download
output_zip = output_filename + '.zip'
with zipfile.ZipFile(output_zip, 'w') as myzip:
    myzip.write(output_filename)
files.download(output_zip)

- Note: with some web browsers, the `files.download()` command won't correctly open a dialog window to download the files.
    - If this happens, check out the "Files" menu on the sidebar
        - can be expanded on the left side of this notebook -- click the > button in the top left-corner to unhide the menu.
    - You can download your file there (and also upload it when you need it in the next notebook).

- You should now have your own file(s) containing Twitter data!

- [-> Next: Content Analysis](https://colab.research.google.com/github/gordeli/NLP_EDHEC/blob/main/colab/04_Content_Analysis.ipynb)